In [3]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import pandas as pd
import time

In [4]:
response = requests.get("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population").text
soup = BeautifulSoup(response, "lxml")
#print(soup.prettify())
data = soup.find('table', {'class': 'wikitable sortable'})
rows = data.find_all('tr')


In [5]:
def getHTMLContent(url):
    html = urlopen(url)
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [6]:
def check(url):
    city = getHTMLContent(url)
    flag = False
    table = city.find('table',{'class':'infobox geography vcard'})
    if table is None:
        flag = True
    return flag

In [7]:
cities = []
names = []
x = ""
for r in rows:
    cells = r.find_all('td')
    if(len(cells)>1):
        city_link = cells[1].find('a')
        x = "https://en.wikipedia.org"+city_link.get('href')
        city_name = city_link.get_text().strip()
        if(check(x)==False):
            cities.append(x)
            names.append(city_name)

In [9]:
def area(url):
    link = ''
    try:
        city = getHTMLContent(url)
        table = city.find('table',{'class':'infobox geography vcard'})
        la = []
        wa = []
        read = False
        for tr in table.find_all('tr'):
            if(tr.get('class') == ['mergedtoprow'] and not read):
                link = tr.find('th')
                if(link!=None):
                    a = link.get_text().strip()
                    if(re.match(r'Area', a)):  
                        read = True
            elif(tr.get('class') == ['mergedrow'] and read==True):
                if(tr.find('th').get_text().strip()=='•\xa0Land'):
                    la.append(tr.find('td').get_text().strip('\n'))
                if(tr.find('th').get_text().strip()=='•\xa0Water'):
                    wa.append(tr.find('td').get_text().strip('\n'))
        return la, wa
    except Exception as error:
        print('Error occured: {}'.format(error))
        return []

In [10]:
def population(url):
    link = ''
    try:
        city = getHTMLContent(url)
        table = city.find('table',{'class':'infobox geography vcard'})
        rank = []
        density = []
        estimate = []
        read = False
        for tr in table.find_all('tr'):
            if(tr.get('class') == ['mergedtoprow'] and not read):
                link = tr.find('th')
                if(link!=None):
                    #print(link.get_text().strip())
                    s = link.get_text().strip()
                    if(re.match(r'Population',s) is not None):  
                        #print('pop')
                        read = True
            elif(tr.get('class') == ['mergedrow'] and read==True):
                if(tr.find('th').get_text().strip()=='•\xa0Rank'):
                    rank.append(tr.find('td').get_text().strip('\n'))
                v =tr.find('th').get_text().strip()
                if(re.match(r'•\xa0Estimate',v) is not None):
                    estimate.append(tr.find('td').get_text().strip('\n'))
                if(tr.find('th').get_text().strip()=='•\xa0Density'):
                    density.append(tr.find('td').get_text().strip('\n'))
        return rank,density,estimate
    except Exception as error:
        print('Error occured: {}'.format(error))
        return []

In [11]:
def time_zones(url):
    link = ''
    try:
        city = getHTMLContent(url)
        table = city.find('table',{'class':'infobox geography vcard'})
        time = []
        read = False
        for tr in table.find_all('tr'):
            if(tr.get('class') == ['mergedtoprow'] and not read):
                link = tr.find('th')
                if(link!=None):
                    #print(link.get_text().strip())
                    if(link.get_text().strip()=='Time zone'):  
                        if(tr.find('a').get_text().strip()=='Time zone'):
                            time.append(tr.find('td').get_text().strip('\n'))
                            read = True
        return time
    except Exception as error:
        print('Error occured: {}'.format(error))
        return []

In [12]:
def state(url):
    link = ''
    try:
        city = getHTMLContent(url)
        table = city.find('table',{'class':'infobox geography vcard'})
        states = []
        read = False
        for tr in table.find_all('tr'):
            if(tr.get('class') == ['mergedtoprow'] and not read):
                link = tr.find('th')
                if(link!=None):
                    a = link.get_text().strip()
                    if(re.match(r'Country', a)):  
                        #print('state')
                        read = True
            elif(tr.get('class') == ['mergedrow'] and read==True):  
                if(tr.find('th').get_text().strip()=='State'):
                    #print("state")
                    #print(tr.find('td').get_text().strip())
                    states.append(tr.find('td').get_text().strip())      
        return states
    except Exception as error:
        print('Error occured: {}'.format(error))
        return []

In [13]:
def elevation(url):
    link = ''
    try:
        city = getHTMLContent(url)
        table = city.find('table',{'class':'infobox geography vcard'})
        ele = []
        read = False
        for tr in table.find_all('tr'):
            if(tr.get('class') == ['mergedtoprow'] and not read):
                link = tr.find('th')
                if(link!=None):
                    b = link.get_text().strip()
                    if(re.match(r'Elevation',b) is not None):  
                        ele.append(tr.find('td').get_text().strip('\n'))
                        read = True
        return ele
    except Exception as error:
        print('Error occured: {}'.format(error))
        return []

In [14]:
def gov(url):
    link = ''
    try:
        city = getHTMLContent(url)
        table = city.find('table',{'class':'infobox geography vcard'})
        typee = []
        body = []
        mayor = []
        read = False
        for tr in table.find_all('tr'):
            if(tr.get('class') == ['mergedtoprow'] and not read):
                link = tr.find('th')
                if(link!=None):
                    #print(link.get_text().strip())
                    s = link.get_text().strip()
                    if(re.match(r'Government',s) is not None):  
                        read = True
            elif(tr.get('class') == ['mergedrow'] and read==True):
                #print(tr.find('th').get_text().strip())
                if(tr.find('th').get_text().strip()=='•\xa0Type'):
                    typee.append(tr.find('td').get_text().strip('\n'))
                if(tr.find('th').get_text().strip()=='•\xa0Body'):
                    body.append(tr.find('td').get_text().strip('\n'))
                if(tr.find('th').get_text().strip()=='•\xa0Mayor'):  
                    mayor.append(tr.find('td').get_text().strip('\n'))
        return typee, body, mayor
    except Exception as error:
        print('Error occured: {}'.format(error))
        return []

In [15]:
def fips_code(url):
    link = ''
    try:
        city = getHTMLContent(url)
        table = city.find('table',{'class':'infobox geography vcard'})
        fips = []
        read = False
        for tr in table.find_all('tr'):
            if(tr.get('class') == ['mergedtoprow'] and not read):
                link = tr.find('th')
                if(link!=None):
                    if(link.get_text().strip()=='FIPS code'):  
                        fips.append(tr.find('td').get_text().strip('\n'))
                        read = True
        return fips
    except Exception as error:
        print('Error occured: {}'.format(error))
        return []

In [16]:
def gather(c):
    temp = []
    la = []
    wa = []
    r = []
    d = []
    e = []
    t = []
    s = []
    el = []
    ty = []
    bo = []
    ma = []
    fc = []
    la, wa = area(c)
    if la == []:
        temp.append("")
    else:
        temp.append(la[0])
        
    if wa == []:
        temp.append("")
    else:
        temp.append(wa[0])
    r,d,e = population(c)
    if r == []:
        temp.append("")
    else:
        temp.append(r[0])
    if d == []:
        temp.append("")
    else:
        temp.append(d[0])
    if e == []:
        temp.append("")
    else:
        temp.append(e[0])
    t = time_zones(c)
    if t == []:
        temp.append("")
    else:
        temp.append(t[0])
    s = state(c)
    if s == []:
        temp.append("")
    else:
        temp.append(s[0])
    el = elevation(c)
    if el == []:
        temp.append("")
    else:
        temp.append(el[0])
    ty, bo, ma = gov(c)
    if ty == []:
        temp.append("")
    else:
        temp.append(ty[0])
    if bo == []:
        temp.append("")
    else:
        temp.append(bo[0])
    if ma == []:
        temp.append("")
    else:
        temp.append(ma[0])
    fc = fips_code(c)
    if fc == []:
        temp.append("")
    else:
        temp.append(fc[0])
    return temp
#print(gather(cities[0]))

In [18]:
data_content = []
t1 = time.time()

for i in range (0, len(cities)):
    temp = []
    temp = gather(cities[i])
    temp.insert(0, names[i])
    #print(temp)
    data_content.append(temp)
    
t2 = time.time()
print("Total execution time is: ", t2-t1)

Total execution time is:  1098.7950241565704


In [19]:
dataset = pd.DataFrame(data_content)
headers = ["City Name","Land area", "Water Area", "Population Rank", "Population Density", "Population Estimate", "Time Zone", "State", "Elevation","Gov_type","Body","Mayor","Fips_code"]

dataset.columns = headers


In [22]:
for column in dataset.columns:
    dataset[column] = dataset[column].str.replace(r"\(.*\)", "")
    dataset[column] = dataset[column].str.replace(r"\[.*\]", "")

In [23]:
dataset['Population Density'] = dataset['Population Density'].str.replace(',', '')
dataset['Population Estimate'] = dataset['Population Estimate'].str.replace(',', '')

In [24]:
dataset.rename(columns = {"Population Density":"Population Density(sq mi)", "Water Area": "Water Area(sq mi)"}, inplace = True)

In [25]:
dataset['Population Density(sq mi)'] = dataset['Population Density(sq mi)'].str.replace(r'[^0-9.]+', '')

In [26]:
for x in range(len(dataset['Water Area(sq mi)'])):
    area = dataset.iloc[x]['Water Area(sq mi)']
    area = area.split()
    if area != []:
        dataset.iloc[x]['Water Area(sq mi)'] = area[0]
    else:
        dataset.iloc[x]['Water Area(sq mi)'] = ""

In [27]:
dataset.to_csv("Final_dataset.csv", index = False)